In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""


In [4]:
usecase_name = "AddQuestion"
project_name = "MatchaEnglishWebsite"
main_flow_prompt="""
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7: Admin presses one word on vocabulary word list.
Step 8: The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9: Admin presses button "Add new question".
Step 10: The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These complusary fields are: one textfield "sentence" for Admins to input sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...',. One textfield 'correct answer' for Admins to input the correct answer word, three textfield 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills in the sentence field by a valid sentence has a symbol '...', a valid sentence must be below 250 characters and must not contains any special character. 
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save' to save the question information.
Step 16: System save the question and redirect Admin to the "Word Information" page of selected word.
"""

alt_prompt=["""
Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.
""",
"""
Alternative flow 2: Admin skip complusary field "sentence".
At step 13 of the basic flow: Admin field all the complusary field except field "sentence".
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
""","""
Alternative flow 3: Admin skip complusary field "'correct answer'.
At step 14 of the basic flow: Admin field all the complusary field except field 'correct answer'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'correct answer' field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
""","""
Alternative flow 4: Admin skip complusary field 'other option 1'.
At step 14 of the basic flow: Admin field all the complusary field except field 'other option 1'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 1' field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
""",
"""
Alternative flow 5: Admin skip complusary field 'other option 2'.
At step 14 of the basic flow: Admin field all the complusary field except field 'other option 2'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 2' field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
""",
"""
Alternative flow 6: Admin skip complusary field 'other option 3'.
At step 14 of the basic flow: Admin field all the complusary field except field 'other option 3'.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 3'field.
Step 17: Admin re-fill the missing field.
Step 18: Admin presses button 'Save'.
""",
"""
Alternative flow 7: Admin choose cancel the question and choose to stay.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Stay'.
Step 16: The dialog closed and the work on that question was kept.
Step 17: Admin presses button 'Save'.
"""
,"""
Alternative flow 8: Admin enters invalid sentences.
At step 13 of the basic flow: Admin fills in the 'sentence' field by an invalid sentence.
Step 14: System show notification that 'Sentence is invalid', prompt Admin to re-enter the "sentence" field.
Step 15: Admin fix the sentence field by a valid sentence.
Step 16: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 17: Admin presses button 'Save'.
"""]

exc_prompt=["""
Exception flow 1: Admin leave by button back on browser without saving the question.
At step 13 of the basic flow: Admin presses the button back on the browser. 
Step 14: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.Question is not saved.
""",
"""
Exception flow 2: Admin choose cancel the question and confirm the cancel.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Confirm'.
Step 16: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.
"""]

prompt_all = """
Main flow: 
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 3: Admin presses one Lesson on the Lesson list.
Step 4: The system redirects Admin to the "Lesson Information" page of selected lesson which show the information of that Lesson and Topic list included in that Lesson.
Step 5: Admin presses one Topic on the Topic list of selected Lesson.
Step 6: The system redirects Admin to the "Topic Information" page of selected topic which show the information of that selected topic and vocabulary word list included in that Topic.
Step 7:Admin presses one word on vocabulary word list.
Step 8:The system redirects Admin to the "Word Information" page of selected word which show the information of that selected word and button "Add new question".
Step 9:Admin presses button "Add new question".
Step 10:The system pop up dialog to choose type of question Admin want to create.
Step 11: Admin presses option add "Create questions for the selection test".
Step 12: The system redirects Admin to the "Add new question for selection test" page of selected word which show all the fields that admin need to input to create a selection test and button 'Save' and 'Cancel'. These compulsory fields are: text field "sentence" for inputting sentence that learner need to fill in blank and the position that the word learner need to fill must present by symbol '...'and "sentence" must not be skip,  a valid sentence must be below 250 characters and not contains any special character. one text field 'correct answer' for Admins to input the correct answer word, three text field 'other option 1', 'other option 2', 'other option 3' for Admins to input the 3 different wrong answer word.
Step 13: Admin fills the field "sentence" with a valid sentence has a symbol '...', 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 14: Admin presses button 'Save' to save the question information.
Step 15: System save the question and redirect Admin to the "Word Information" page of selected word.

Alternative flow: 
Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.

Alternative flow 2: Admin skip compulsory field "sentence".
At step 13 of the basic flow: Admin field all the compulsory field except field "sentence".
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 3: Admin skip compulsory field "'correct answer'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'correct answer'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'correct answer' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 4: Admin skip compulsory field 'other option 1'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 1'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 1' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 5: Admin skip compulsory field 'other option 2'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 2'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 2' field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 6: Admin skip compulsory field 'other option 3'.
At step 13 of the basic flow: Admin field all the compulsory field except field 'other option 3'.
Step 14: Admin presses button 'Save'.
Step 15: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing 'other option 3'field.
Step 16: Admin re-fill the missing field.
Step 17: Admin presses button 'Save'.

Alternative flow 7: Admin choose cancel the question and choose to stay.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Stay'.
Step 16: The dialog closed and the work on that question was kept.
Step 17: Admin presses button 'Save'.

Alternative flow 8: Admin enters invalid sentences.
At step 13 of the basic flow: Admin fills all the other compulsory field but fills in the 'sentence' field by an invalid sentence.
Step 14: System show notification that 'Sentence is invalid', prompt Admin to re-enter the "sentence" field.
Step 16: Admin re-enter the missing field.
Step 17: Admin presses button 'Save'.

Exception flow: 
Exception flow 1: Admin leave by button back on browser without saving the question.
At step 13 of the basic flow: Admin presses the button back on the browser. 
Step 14: The system redirects Admin to the "Word Information" page of selected word and Question is not saved. Question is not saved.

Exception flow 2: Admin choose cancel the question and confirm the cancel.
At step 13 of the basic flow: Admin presses button 'Cancel'.
Step 14: System show dialog 'Are you sure you want to cancel the question?' with 2 option buttons 'Confirm' and 'Stay'.
Step 15: Admin presses the buttons 'Confirm'.
Step 16: The system redirects Admin to the "Word Information" page of selected word and Question is not saved.
"""

In [5]:
for alt in alt_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+alt}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(alt)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")
for exc in exc_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+exc}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(exc)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")

  print()
  print()
promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
sub_gpt_response = ask(promptSubScenario, client, model)
print(exc)
print("all flow gen sub flow:"+sub_gpt_response +"\n")


Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.


sub_gpt_response:Based on the provided main flow and the alternative flow, here are the scenarios that can lead the user from the main flow to change to the alternative or exception flow:

1. **Admin Accesses Lesson Management from Homepage**


Alternative flow 2: Admin skip complusary field "sentence".
At step 13 of the basic flow: Admin field all the complusary field except field "sentence".
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 17: Admin re-

In [6]:
for alt in alt_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+alt}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(alt)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")
for exc in exc_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+exc}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(exc)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")

  print()
  print()
promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
sub_gpt_response = ask(promptSubScenario, client, model)
print(exc)
print("\all flow gen sub flow:"+sub_gpt_response +"\n")


Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.


sub_gpt_response:Based on the provided main flow and the alternative flow, here are the scenarios that can lead the user from the main flow to change to the alternative or exception flow mentioned:

1. **Admin Accesses Lesson Management from Homepage**


Alternative flow 2: Admin skip complusary field "sentence".
At step 13 of the basic flow: Admin field all the complusary field except field "sentence".
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 17:

In [7]:
for alt in alt_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+alt}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(alt)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")
for exc in exc_prompt:
  promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt +"\n"+exc}
    ]
  sub_gpt_response = ask(promptSubScenario, client, model)
  print(exc)
  print("\nsub_gpt_response:"+sub_gpt_response +"\n")

  print()
  print()
promptSubScenario = [
      { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
sub_gpt_response = ask(promptSubScenario, client, model)
print(exc)
print("\all flow gen sub flow:"+sub_gpt_response +"\n")


Alternative flow 1: Admin go to "Lesson Management" page from Homepage.
At step 1 of the basic flow: Admin presses the "Lesson Management" tab at Homepage.
Go back to step 2 in the basic flow and continue with the steps from step 2.


sub_gpt_response:Based on the provided information, here are the scenarios that can lead the user from the main flow to the alternative or exception flow:

1. **Admin Accesses Lesson Management from Homepage**


Alternative flow 2: Admin skip complusary field "sentence".
At step 13 of the basic flow: Admin field all the complusary field except field "sentence".
Step 14: Admin fills 'correct answer' with the correct word form of the blank in the sentence, 'other option 1', 'other option 2', 'other option 3' with 3 different wrong answer word.
Step 15: Admin presses button 'Save'.
Step 16: System show notification that 'Admin did not fill all the fields', prompt Admin to fill the missing "sentence" field.
Step 17: Admin re-fill the missing field.
Step 18: 